In [1]:
import os
import json
import torch
from torch import nn
from torch.utils.data import Dataset
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
from transformers import (
    CLIPProcessor,
    CLIPModel,
    Trainer,
    TrainingArguments,
    DefaultDataCollator,
)
from tqdm import tqdm

In [2]:
os.environ["WANDB_DISABLED"] = "true"

In [3]:
# 配置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device=}")

Using device=device(type='cuda')


In [4]:
images_folder = "dataset_spain"  # 保存西班牙图片的文件夹路径
labels_json = "dataset_spain/comunidad.json"  # 包含正确标签的 JSON 文件路径

In [5]:
# 西班牙自治区列表
COMUNIDADES = [
    "Andalucía",
    "Aragón",
    "Asturias",
    "Canary Is.",
    "Cantabria",
    "Castilla-La Mancha",
    "Castilla y León",
    "Cataluña",
    "Valenciana",
    "Extremadura",
    "Galicia",
    "Islas Baleares",
    "La Rioja",
    "Madrid",
    "Murcia",
    "País Vasco",
    "Navarra",
    "Ceuta",
    "Melilla",
]

In [6]:
# 定义计算预测函数
def predict_comunidad(model,processor,image_path):
    """
    使用 CLIP 模型预测图片所属的 Comunidad Autónoma
    :param image_path: 图片路径
    :return: 最可能的 Comunidad 名称
    """
    image = Image.open(image_path)
    inputs = processor(
        text=COMUNIDADES, images=image, return_tensors="pt", padding=True
    )

    # 将输入移动到 GPU
    inputs = {key: value.to(device) for key, value in inputs.items()}

    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1)
    predicted_idx = probs.argmax(dim=1).item()
    return COMUNIDADES[predicted_idx]


def calculate_accuracy(model, processor, images_folder, labels_json):
    """
    计算模型在西班牙图片上的预测准确率
    :param images_folder: 图片文件夹路径
    :param labels_json: 包含正确标签的 JSON 文件路径
    :return: 准确率
    """
    # 加载标签
    with open(labels_json, "r", encoding="utf-8") as f:
        labels = json.load(f)

    correct = 0
    total = 0

    # 遍历文件夹中的图片
    for image_id, true_comunidad in tqdm(labels.items(), desc="Processing images"):
        image_path = os.path.join(images_folder, f"{image_id}.png")
        if os.path.exists(image_path):
            predicted_comunidad = predict_comunidad(model, processor, image_path)
            # print(f"Image {image_id}: Predicted {predicted_comunidad}, True {true_comunidad}")
            if predicted_comunidad == true_comunidad:
                correct += 1
            total += 1

    accuracy = correct / total if total > 0 else 0
    # print(f"Total images processed: {total}")
    # print(f"Accuracy: {accuracy:.2%}")
    return accuracy

In [7]:
# 定义数据集类
class ImageTextDataset(Dataset):
    def __init__(self, image_paths, labels, processor, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.processor = processor
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # 获取图像和文本
        image_path = self.image_paths[idx]
        label = self.labels[idx]

        # 加载图像
        image = Image.open(image_path).convert("RGB")  # 转为 RGB（去除透明度）

        if self.transform:
            image = self.transform(image)

        # 使用处理器对文本和图像进行预处理
        inputs = self.processor(
            text=label,
            images=image,
            return_tensors="pt",
            padding="max_length",  # 一定要开启这个，不然会因为文本长度不同而报错
            do_rescale=False,
        )

        inputs["pixel_values"] = inputs["pixel_values"].squeeze(0)  # 移除多余的批次维度
        inputs["attention_mask"] = inputs["attention_mask"].squeeze(0)

        return inputs


# 图像标准化和数据增强（如果需要）
transform = transforms.Compose(
    [
        # transforms.Resize((336, 336)),  # 调整图像大小为 336x336
        transforms.ToTensor(),
    ]
)


# 定义对比损失（contrastive loss）
class CLIPContrastiveLoss(nn.Module):
    def __init__(self, temperature=0.07):
        super().__init__()
        self.temperature = temperature

    def forward(self, logits_per_image, logits_per_text):
        # 对 logits 进行归一化（计算余弦相似度）
        logits_per_image = logits_per_image / self.temperature
        logits_per_text = logits_per_text / self.temperature

        # 计算对比损失
        labels = torch.arange(logits_per_image.size(0)).to(logits_per_image.device)
        loss_img = F.cross_entropy(logits_per_image, labels)
        loss_txt = F.cross_entropy(logits_per_text, labels)

        # 返回两个损失的平均值
        return (loss_img + loss_txt) / 2


# 定义自定义的 Trainer 类，使用我们定义的对比损失函数
class CLIPTrainer(Trainer):
    def __init__(self, *args, contrastive_loss_fn=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.contrastive_loss_fn = contrastive_loss_fn

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # 前向传播
        outputs = model(**inputs)
        logits_per_image = outputs.logits_per_image
        logits_per_text = outputs.logits_per_text

        # 计算对比损失
        loss = self.contrastive_loss_fn(logits_per_image, logits_per_text)

        return (loss, outputs) if return_outputs else loss

In [8]:
def get_train_dataset(processor):
    # 加载社区标签数据
    with open(labels_json, "r") as f:
        comunidad_data = json.load(f)

    # 图像路径和标签的配对
    image_paths = []
    labels = []

    for img_id, label in comunidad_data.items():
        img_path = os.path.join(images_folder, f"{img_id}.png")
        if os.path.exists(img_path):
            image_paths.append(img_path)
            labels.append(label)

    # 创建数据集和数据加载器
    train_dataset = ImageTextDataset(image_paths, labels, processor, transform=transform)
    return train_dataset

In [9]:
def train(epoch:int,batch_size:int,learning_rate:float,dropout:float,epochs_to_log:list,log_list:list):
    # 加载预训练的 CLIP 模型和处理器
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    model.vision_model.embeddings.dropout = nn.Dropout(
        p=dropout
    )  # 设置 30% 的 dropout 概率
    model.text_model.embeddings.dropout = nn.Dropout(
        p=dropout
    )  # 设置 30% 的 dropout 概率
    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=1,  # 总训练 epoch 数
        per_device_train_batch_size=batch_size,  # 每个设备的批量大小
        # logging_dir="./logs",                # 日志目录
        logging_strategy="epoch",  # 按 epoch 记录日志
        # save_strategy="epoch",               # 按 epoch 保存模型
        logging_dir=None,  # 不记录日志
        # logging_strategy="no",               # 不记录日志
        save_strategy="no",  # 不保存模型
        evaluation_strategy="epoch",  # 按 epoch 进行评估
        save_total_limit=3,  # 最多保存3个模型
        learning_rate=learning_rate,  # 显式设置学习率
    )

    train_dataset=get_train_dataset(processor)

    # 定义训练参数和 Trainer
    contrastive_loss_fn = CLIPContrastiveLoss()

    data_collator = DefaultDataCollator(return_tensors="pt")

    for epoch in range(1, epoch+1):

        trainer = CLIPTrainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=train_dataset,
            tokenizer=processor,
            contrastive_loss_fn=contrastive_loss_fn,
            data_collator=data_collator,  # Add this line
        )
        model.train()
        trainer.train()
        if epoch in epochs_to_log:

          model.eval()
          acc = calculate_accuracy(model, processor, images_folder, labels_json)

          print(f"{epoch=} {batch_size=} {learning_rate=} {dropout=} {acc=}")
          log_list.append([learning_rate, dropout, epoch, acc])

          torch.cuda.empty_cache()
          del trainer

In [14]:


log_list=[]

epochs_to_log=[40,45,50,55,60]
for learning_rate in [5e-6,1e-6]:
    for dropout in [0,0.1,0.3]:
        train(epoch=60,batch_size=77,learning_rate=learning_rate,dropout=dropout,epochs_to_log=epochs_to_log,log_list=log_list)

# 保存模型
# model.save_pretrained("./clip_model")
# processor.save_pretrained("./clip_processor")
print("training done")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-7-56db554ea82b>:70: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CLIPTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,61.856300,No log


Epoch,Training Loss,Validation Loss
1,40.260600,No log


Epoch,Training Loss,Validation Loss
1,28.606400,No log


Epoch,Training Loss,Validation Loss
1,31.803500,No log


...

Epoch,Training Loss,Validation Loss
1,3.733200,No log


Epoch,Training Loss,Validation Loss
1,3.475100,No log


Epoch,Training Loss,Validation Loss
1,3.805400,No log


Processing images: 100%|██████████| 77/77 [00:02<00:00, 35.58it/s]


epoch=60 batch_size=77 learning_rate=1e-06 dropout=0.3 acc=0.4155844155844156
training done


In [15]:
print(json.dumps(log_list))

[[5e-06, 0, 40, 0.8441558441558441], [5e-06, 0, 45, 0.8181818181818182], [5e-06, 0, 50, 0.7532467532467533], [5e-06, 0, 55, 0.7272727272727273], [5e-06, 0, 60, 0.6363636363636364], [5e-06, 0.1, 40, 0.8441558441558441], [5e-06, 0.1, 45, 0.8181818181818182], [5e-06, 0.1, 50, 0.7532467532467533], [5e-06, 0.1, 55, 0.7272727272727273], [5e-06, 0.1, 60, 0.6363636363636364], [5e-06, 0.3, 40, 0.8441558441558441], [5e-06, 0.3, 45, 0.8181818181818182], [5e-06, 0.3, 50, 0.7532467532467533], [5e-06, 0.3, 55, 0.7272727272727273], [5e-06, 0.3, 60, 0.6363636363636364], [1e-06, 0, 40, 0.4025974025974026], [1e-06, 0, 45, 0.23376623376623376], [1e-06, 0, 50, 0.4025974025974026], [1e-06, 0, 55, 0.37662337662337664], [1e-06, 0, 60, 0.4155844155844156], [1e-06, 0.1, 40, 0.4025974025974026], [1e-06, 0.1, 45, 0.23376623376623376], [1e-06, 0.1, 50, 0.4025974025974026], [1e-06, 0.1, 55, 0.37662337662337664], [1e-06, 0.1, 60, 0.4155844155844156], [1e-06, 0.3, 40, 0.4025974025974026], [1e-06, 0.3, 45, 0.23376623

In [16]:
for log in log_list:
  print(log)

[5e-06, 0, 40, 0.8441558441558441]
[5e-06, 0, 45, 0.8181818181818182]
[5e-06, 0, 50, 0.7532467532467533]
[5e-06, 0, 55, 0.7272727272727273]
[5e-06, 0, 60, 0.6363636363636364]
[5e-06, 0.1, 40, 0.8441558441558441]
[5e-06, 0.1, 45, 0.8181818181818182]
[5e-06, 0.1, 50, 0.7532467532467533]
[5e-06, 0.1, 55, 0.7272727272727273]
[5e-06, 0.1, 60, 0.6363636363636364]
[5e-06, 0.3, 40, 0.8441558441558441]
[5e-06, 0.3, 45, 0.8181818181818182]
[5e-06, 0.3, 50, 0.7532467532467533]
[5e-06, 0.3, 55, 0.7272727272727273]
[5e-06, 0.3, 60, 0.6363636363636364]
[1e-06, 0, 40, 0.4025974025974026]
[1e-06, 0, 45, 0.23376623376623376]
[1e-06, 0, 50, 0.4025974025974026]
[1e-06, 0, 55, 0.37662337662337664]
[1e-06, 0, 60, 0.4155844155844156]
[1e-06, 0.1, 40, 0.4025974025974026]
[1e-06, 0.1, 45, 0.23376623376623376]
[1e-06, 0.1, 50, 0.4025974025974026]
[1e-06, 0.1, 55, 0.37662337662337664]
[1e-06, 0.1, 60, 0.4155844155844156]
[1e-06, 0.3, 40, 0.4025974025974026]
[1e-06, 0.3, 45, 0.23376623376623376]
[1e-06, 0.3, 50,